In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Minimax
A program to play a game of chess using the Minimax algorithm. The Minimax algorithm calculates all different possibilities of the current position and chooses the best move. It assumes that all players try to play the best possible moves. From whites perspective white chooses the best move of blacks best moves. We have to limit the amout of moves that minimax will go into the future due to the huge amount of possibilities that chess has. Therefore we stop the minimax algorithm at a certain depth.  

In [2]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random

# Pkg.add("NBInclude")
using NBInclude

In [3]:
@nbinclude("EvaluatePosition.ipynb")

keine Figur vorhandenSubCapture

100000

In [4]:
gCacheMaxVal = Dict()

Dict{Any, Any}()

In [5]:
gCacheMinVal = Dict()

Dict{Any, Any}()

## MaxValue-function
This function looks for the maximum value that white can achieve from a position. Meanwhile black tries to minimize the value of the position. Therefore white will take the maximal value of the minimum value of the next move that can happen.

Note: The depth is subtracted to give a better/worse rating to evaluate short mating paths as a petter path. (if there is a Mate in 3 (M3) and M1 Path then choose the M1 Path)

`maxValue(State::Board, depth::Int64)` is the non-incremental implementation of the maxValue function. It takes in the current board and the depth it should calculate and returns the best score it can achieve.

In [6]:
function maxValue(State::Board, depth::Int64)
    if isterminal(State) || depth == 0
        return evaluate_position(State) - depth
    end
    return maximum([ minValue(domove(State, ns), depth-1) for ns in moves(State) ])
end

maxValue (generic function with 1 method)

`maxValue(State::Board, score, depth::Int64)` is the incremental implementation of the maxValue function. It takes in the current board, the current score of the board and the depth it should calculates and returns the best score it can achieve.

In [7]:
function maxValue(State::Board, score::Int64, depth::Int64)
    if isterminal(State) 
        return terminal_evaluation(State) - depth
    elseif depth <= 1
        return maximum([ evaluate_move(State, move, score) for move in moves(State) ])
    end
    return maximum([ minValue(domove(State, move), evaluate_move(State, move, score), depth-1) for move in moves(State) ])
end

maxValue (generic function with 2 methods)

### with Memoization

This function is the same maxValue function above. Additionally it will save the calculated score in a global Cache. It will retrieve the value if it has already been calculated. It therefore must already be in the cache.

In [8]:
function maxValue(State::Board, score::Int64, depth::Int64)
    # Memoization
    global gCacheMaxVal
    entry = (State, score, depth)
    if entry in keys(gCacheMaxVal)
        return gCacheMaxVal[entry]
    end
    # maxValue Calculation
    # run old minValue result = minValue(State, score, depth)
    if isterminal(State) 
        result = terminal_evaluation(State) - depth
    elseif depth <= 1
        result = maximum([ evaluate_move(State, move, score) for move in moves(State) ])
    else
        result = maximum([ minValue(domove(State, move), evaluate_move(State, move, score), depth-1) for move in moves(State) ])
    end
    # Save in Cache
    merge!(gCacheMaxVal, Dict(entry => result))
    result
end

maxValue (generic function with 2 methods)

## MinValue-function
MinValue does the opposite of the MaxValue function. Therefore it looks for the minimum value that black can achieve from a position. Meanwhile white tries to maximize the value of the position. Therefore black will take the minimal value of the maximum value of the next move that can happen.

In [9]:
function minValue(State::Board, depth::Int64)
    if isterminal(State) || depth == 0
      return evaluate_position(State) + depth
    end
    return minimum([ maxValue(domove(State, ns), depth-1) for ns in moves(State) ])
end

minValue (generic function with 1 method)

In [10]:
function minValue(State::Board, score::Int64, depth::Int64)
    if isterminal(State) 
        return terminal_evaluation(State) + depth
    elseif depth <= 1
        return minimum([ evaluate_move(State, move, score) for move in moves(State) ])
    end
    return minimum([ maxValue(domove(State, move), evaluate_move(State, move, score), depth-1) for move in moves(State) ])
end

minValue (generic function with 2 methods)

### with Memoization

In [11]:
function minValue(State::Board, score::Int64, depth::Int64)
    # Memoization
    global gCacheMinVal
    entry = (State, score, depth)
    if entry in keys(gCacheMinVal)
        return gCacheMinVal[entry]
    end
    # maxValue Calculation
    # run old minValue result = minValue(State, score, depth)
    if isterminal(State) 
        result = terminal_evaluation(State) - depth
    elseif depth <= 1
        result = minimum([ evaluate_move(State, move, score) for move in moves(State) ])
    else
        result = minimum([ maxValue(domove(State, move), evaluate_move(State, move, score), depth-1) for move in moves(State) ])
    end
    # Save in Cache
    merge!(gCacheMinVal, Dict(entry => result))
    result
end

minValue (generic function with 2 methods)

In [12]:
methods(minValue)

# 2 methods for generic function "minValue":
[1] minValue(State::Board, depth::Int64) in Main at In[9]:1
[2] minValue(State::Board, score::Int64, depth::Int64) in Main at In[11]:1

In [13]:
methods(maxValue)

# 2 methods for generic function "maxValue":
[1] maxValue(State::Board, depth::Int64) in Main at In[6]:1
[2] maxValue(State::Board, score::Int64, depth::Int64) in Main at In[8]:1

In [ ]:
b = fromfen("4k3/8/8/4p3/8/2BK4/8/q7 w - - 0 1")
minValue(b, 3)

In [14]:
b = fromfen("4k3/8/8/4p3/8/2BK4/8/q7 w - - 0 1")
minValue(b, evaluate_position(b), 3)

-710

In [16]:
gCacheMinVal

Dict{Any, Any} with 289 entries:
  (Board (4k3/q7/8/4p3/8/2B5/2K5/8 w - -):…                             => -700
  (Board (8/5k2/8/B3p3/8/3K4/8/q7 w - -):…                              => -720
  (Board (4k3/8/8/8/4p3/2B5/4K3/q7 w - -):…                             => -695
  (Board (4k3/8/8/4p3/8/3K4/3B4/2q5 w - -):…                            => -720
  (Board (4k3/8/8/4p3/8/2B5/2K5/6q1 w - -):…                            => -700
  (Board (q3k3/8/8/4p3/8/2B5/3K4/8 w - -):…                             => -690
  (Board (4k3/8/8/4p3/8/3K4/8/4q3 w - -):…                              => -1045
  (Board (4k3/8/8/4p3/8/2B5/3K4/2q5 w - -):…                            => -690
  (Board (3k4/8/8/4p3/3B4/3K4/8/q7 w - -):…                             => -700
  (Board (3k4/8/8/4p3/8/3K4/1B6/q7 w - -):…                             => -705
  (Board (4k3/8/8/4p3/8/2B5/3K4/1q6 w - -):…                            => -700
  (Board (4k3/8/8/4p3/8/q2K4/1B6/8 w - -):…                             => -715
  (Boa

## Add Memoization

In [ ]:
# @nbinclude("Memoization.ipynb")

Using `invoke` to overwrite the functions defined above. Q: https://discourse.julialang.org/t/overwriting-functions/404

In [ ]:
# maxValue(State::Board, score::Int64, depth::Int64) = invoke(memoize(maxValue(State::Board, score::Int64, depth::Int64), Tuple{Board, Int64, Int64}, State, score, depth))

In [ ]:
# minValue(State::Board, score::Int64, depth::Int64) = invoke(memoize(minValue(State::Board, score::Int64, depth::Int64), Tuple{Board, Int64, Int64}, State, score, depth))

In [ ]:
# methods(maxValue)

In [ ]:
# b = fromfen("k7/2K5/8/8/8/8/1R6/8 b - - 0 1")
# maxValue(b, 440, 5)

## Minimax-function

The `minimax` function takes in a board and returns the value and the board of the best move. It's depth must be set.

non-inkremental implementation

In [17]:
function minimax(State::Board, depth::Int64)
    next_moves = moves(State)
    if sidetomove(State) == WHITE
        bestVal = maxValue(State, depth)
        BestMoves = [move for move in next_moves if minValue(domove(State, move), depth-1) == bestVal]
    elseif sidetomove(State) == BLACK
        bestVal = minValue(State, depth)
        BestMoves = [move for move in next_moves if maxValue(domove(State, move), depth-1) == bestVal]
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

minimax (generic function with 1 method)

The `minimax_verbal` function prints more information. 

In [18]:
function minimax(State::Board, score::Int64, depth::Int64)
    next_moves = moves(State)
    print(next_moves)
    BestMoves = []
    bestVal = 0
    if sidetomove(State) == WHITE
        bestVal = maxValue(State, score, depth)
        BestMoves = [move for move in next_moves if minValue(domove(State, move), evaluate_move(State, move, score), depth-1) == bestVal]
    elseif sidetomove(State) == BLACK
        bestVal = minValue(State, score, depth)
        BestMoves = [move for move in next_moves if maxValue(domove(State, move), evaluate_move(State, move, score), depth-1) == bestVal]
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

minimax (generic function with 2 methods)

In [19]:
function minimax_verbal(State::Board, score::Int64, depth::Int64)
    next_moves = moves(State)
    BestMoves = []
    pprint(State, color = true)
    if sidetomove(State) == WHITE
        bestVal = maxValue(State, score, depth)
        for move in next_moves
            state_after_move = domove(State, move)
            minVal = minValue(state_after_move, evaluate_move(State, move, score), depth-1)
            println("-----------------------")
            println(move)
            display(state_after_move)
            println(minVal)
            println()
            if minVal == bestVal
                append!(BestMoves,  [move])
            end
        end
    elseif sidetomove(State) == BLACK
        bestVal = minValue(State, score, depth)
        for move in next_moves
            state_after_move = domove(State, move)
            maxVal = maxValue(state_after_move, evaluate_move(State, move, score), depth-1)
            println("-----------------------")
            println(move)
            display(state_after_move)
            println(minVal)
            println()
            if minVal == bestVal
                append!(BestMoves,  [move])
            end
        end
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

minimax_verbal (generic function with 1 method)

In [20]:
function playMinimaxMove(game::Game, depth::Int64)
    minimaxeval = minimax(board(game), depth)
    domove!(game, minimaxeval[2])
    return minimaxeval[1]
end

playMinimaxMove (generic function with 1 method)

In [21]:
function playMinimaxMove(game::Game, depth::Int64, score::Int64)
    minimaxeval = minimax(board(game), score, depth)
    domove!(game, minimaxeval[2])
    return minimaxeval[1]
end

playMinimaxMove (generic function with 2 methods)

In [22]:
b = fromfen("k7/2K5/8/8/8/8/1R6/8 b - - 0 1")
current_board_value = evaluate_position(b)
println(current_board_value)
@time begin
    best_move = minimax(b, current_board_value, 5)
    print(best_move)
end

440
Move[Move(a8a7)](99999, Move(a8a7))  0.330165 seconds (520.85 k allocations: 31.052 MiB, 3.83% gc time, 71.56% compilation time)


In [24]:
length(gCacheMinVal)

1654

In [25]:
b = fromfen("8/4k3/8/4p3/8/2BK4/8/q7 w - - 0 1")
current_board_value = evaluate_position(b)
println(current_board_value)
@time begin
    best_move = minimax(b, 3)
    print(best_move)
end

-680
(185, Move(c3a1))  0.277841 seconds (2.47 M allocations: 94.970 MiB, 7.76% gc time, 28.68% compilation time)


In [28]:
b = fromfen("8/4k3/8/4p3/8/2BK4/8/q7 w - - 0 1")
current_board_value = evaluate_position(b)
println(current_board_value)
@time begin
    best_move = minimax_verbal(b, current_board_value, 5)
    print(best_move)
end

-680
                        
             k          
                        
             p          
                        
       b  k             
                        
 q                      
8/4k3/8/4p3/8/2BK4/8/q7 w - -
-----------------------
Move(c3a5)
                        
             k          
                        
 b           p          
                        
          k             
                        
 q                      
8/4k3/8/B3p3/8/3K4/8/q7 b - -
-1010

-----------------------
Move(c3a1)
                        
             k          
                        
             p          
                        
          k             
                        
 b                      
8/4k3/8/4p3/8/3K4/8/B7 b - -
190

-----------------------
Move(c3b4)
                        
             k          
                        
             p          
    b                   
          k             
                        


 q                      
8/4k3/8/4p3/1B6/3K4/8/q7 b - -
-690

-----------------------
Move(c3b2)
                        
             k          
                        
             p          
                        
          k             
    b                   
 q                      
8/4k3/8/4p3/8/3K4/1B6/q7 b - -
-1050

-----------------------
Move(c3d4)
                        
             k          
                        
             p          
          b             
          k             
                        
 q                      
8/4k3/8/4p3/3B4/3K4/8/q7 b - -
-1025

-----------------------
Move(c3d2)
                        
             k          
                        
             p          
                        
          k             
          b             
 q                      
8/4k3/8/4p3/8/3K4/3B4/q7 b - -
-690

-----------------------
Move(c3e5)
                        
             k          
                        
          


-----------------------
Move(c3e1)
                        
             k          
                        
             p          
                        
          k             
                        
 q           b          
8/4k3/8/4p3/8/3K4/8/q3B3 b - -
-1010

-----------------------
Move(d3c4)
                        
             k          
                        
             p          
       k                
       b                
                        
 q                      
8/4k3/8/4p3/2K5/2B5/8/q7 b - -
-695

-----------------------
Move(d3c2)
                        
             k          
                        
             p          
                        
       b                
       k                
 q                      
8/4k3/8/4p3/8/2B5/2K5/q7 b - -
-665

-----------------------
Move(d3d2)
                        
             k          
                        
             p          
                        
       b              


-----------------------
Move(d3e4)
                        
             k          
                        
             p          
             k          
       b                
                        
 q                      
8/4k3/8/4p3/4K3/2B5/8/q7 b - -
-1065

-----------------------
Move(d3e3)
                        
             k          
                        
             p          
                        
       b     k          
                        
 q                      
8/4k3/8/4p3/8/2B1K3/8/q7 b - -
-1005

-----------------------
Move(d3e2)
                        
             k          
                        
             p          
                        
       b                
             k          
 q                      
8/4k3/8/4p3/8/2B5/4K3/q7 b - -
-1005

(190, Move(c3a1)) 32.181087 seconds (71.31 M allocations: 3.265 GiB, 6.93% gc time)
